In [1]:
#Street Easy Scrape Because Whynot

import pandas as pd

import requests
from bs4 import BeautifulSoup

In [2]:
def get_main_listings(page):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
    url = 'https://www.renthop.com/search/nyc?max_price=50000&min_price=0&sort=hopscore&q=&search=0&page={}'
    page = requests.get(url, headers=headers, timeout=50) #it will keep trying at this stage, the loop will not continue to iterate until a response is given
    c = page.content
    return BeautifulSoup(c, 'html.parser')
    

In [3]:
get_main_listings(1)

<!DOCTYPE doctype html>

<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="en" http-equiv="Content-Language"/>
<title>New York Apartment Finder, NYC Apartments Search</title>
<meta content="Newly listed NYC apartments for rent. Smooth work commute, popular bars and nightlife, nearby restaurants and grocery stores, and safety. Find your perfect home in New York, NY." name="description"/>
<meta content="RentHop" name="author"/>
<meta content="Copyright (c) 2009 - 2019 RentHop.com" name="Copyright"/>
<meta content="124300320712" property="fb:page_id"/>
<meta content="294321126236" property="fb:app_id"/>
<meta content="https://www.renthop.com/images/renthop_icon_small.png" name="og:image"/>
<link href="https://www.renthop.com/images/renthop_icon_small.png" rel="image_src"/>
<meta content="New York Apartment Finder, NYC Apartments Search" name="og:title"/>
<meta content="Newly listed NYC apartments for rent. Smooth work commute, popu

In [4]:
one_listing = get_main_listings(1).find_all('div', class_ = 'search-listing font-size-10 my-4 my-md-0 py-0 py-md-4')[0]
one_listing

<div class="search-listing font-size-10 my-4 my-md-0 py-0 py-md-4" latitude="40.7668" listing_id="14417572" longitude="-73.9256">
<div class="search-thumbs">
<a href="https://www.renthop.com/listings/30-59-crescent-street/3r/14417572" style="text-decoration: none">
<img alt="2 Bedrooms, Astoria Rental in NYC for $2,550 - Photo 1" class="search-thumb" id="listing-14417572-img" src="https://photos.renthop.com/p/s/640x640/14417572_6279e408edae0dea252a5e0c634266a3.jpg" style="vertical-align: top;"/>
</a>
<a class="d-none d-xl-inline-block" href="https://www.renthop.com/listings/30-59-crescent-street/3r/14417572" style="text-decoration: none">
<img alt="2 Bedrooms, Astoria Rental in NYC for $2,550 - Photo 2" class="search-thumb" src="https://photos.renthop.com/p/s/640x640/14417572_422df4a26ef791dc1d96e91e1ee532ee.jpg" style="vertical-align: top;"/>
</a>
</div>
<div class="search-info pr-3 pl-3 pr-md-0 pl-md-4">
<div>
<div class="float-right align-top font-size-9">
<span class="font-gray-2">

In [5]:
one_page = get_main_listings(1).find_all('div', class_ = 'search-listing font-size-10 my-4 my-md-0 py-0 py-md-4')

In [6]:
one_listing.find('a', href=True)['href']

'https://www.renthop.com/listings/30-59-crescent-street/3r/14417572'

In [7]:
one_listing.find('a', class_ = 'font-size-11 listing-title-link b').get_text()


'30-59 Crescent Street, Apt 3R'

In [8]:
one_listing.find("div", {"class": "font-size-9 overflow-ellipsis"}).get_text().replace("\n","")

'Astoria, Northwestern Queens, Queens'

In [9]:
one_listing.find("td", {"class": "font-size-11 bold"}).get_text().replace("\n","")

'$2,550'

In [10]:
one_listing.find("td", {"class": "font-size-11 bold"}).find_next().get_text().replace("\n","")

'2 Bed'

In [11]:
one_listing.find("td", {"class": "font-size-11 bold"}).find_next_sibling().find_next_sibling().get_text().replace("\n","")

'1 Bath'

In [12]:
one_listing.find_all("div", {"class": "font-size-9"})[-1].get_text().replace("\n","")

'Exclusive\xa0650 ft²'

In [87]:
def get_main_listing_dict(listing):
    
    #Creates an initial dictionary with neat data
    
    listing_dict = {
        'url' : listing.find('a', href=True)['href'],
        'rent' : listing.find("td", {"class": "font-size-11 bold"}).get_text().replace("\n","").replace("$","").replace(",",""),
        'bedrooms' : listing.find("td", {"class": "font-size-11 bold"}).find_next().get_text().replace("\n","").replace(" Bed",""),
        'bath' : listing.find("td", {"class": "font-size-11 bold"}).find_next_sibling().find_next_sibling().get_text().replace("\n","").replace(" Bath","")
    }
    
    #Separates the address and unit number
    try:
        listing_dict['address'] = listing.find('a', class_ = 'font-size-11 listing-title-link b').get_text().split(', ')[0]
        listing_dict['unit'] = listing.find('a', class_ = 'font-size-11 listing-title-link b').get_text().split(', ')[1]
    except:
        listing_dict['address'] = listing.find('a', class_ = 'font-size-11 listing-title-link b').get_text().split(', ')
        
    
    #Separates the borough
    listing_dict['borough'] = listing.find("div", {"class": "font-size-9 overflow-ellipsis"}).get_text().replace("\n","").split(', ')[-1]
    listing_dict['neighborhood'] = listing.find("div", {"class": "font-size-9 overflow-ellipsis"}).get_text().replace("\n","").split(', ')[0]
    
    #Cleans the amenities data in the 'extras' column
    try:
        extra = listing.find_all("div", {"class": "font-size-9"})[-1].get_text().replace("\n","")
        listing_dict['extra'] = extra.replace("\xa0"," · ")

        
        try:
            listing_dict['extra'] = listing_dict['extra'].split(' · ')
        except:
            listing_dict['extra'] = [listing_dict['extra']]
    except:
        listing_dict['extra'] = [listing_dict['extra']]
        
    
    try:
        listing_dict['sqft'] = list(filter(lambda x: ' ft²' in x, listing_dict['extra']))[0].split(' ')[0]
        listing_dict['sqft'] = int(listing_dict['sqft'])
    except:
        listing_dict['sqft'] = 0
        
        
        
        
        
        
    return listing_dict
    
    

In [88]:
get_main_listing_dict(one_listing)

{'url': 'https://www.renthop.com/listings/30-59-crescent-street/3r/14417572',
 'rent': '2550',
 'bedrooms': '2',
 'bath': '1',
 'address': '30-59 Crescent Street',
 'unit': 'Apt 3R',
 'borough': 'Queens',
 'neighborhood': 'Astoria',
 'extra': ['Exclusive', '650 ft²'],
 'sqft': 650}

In [89]:
list(map(lambda x: get_main_listing_dict(x), one_page ))

[{'url': 'https://www.renthop.com/listings/30-59-crescent-street/3r/14417572',
  'rent': '2550',
  'bedrooms': '2',
  'bath': '1',
  'address': '30-59 Crescent Street',
  'unit': 'Apt 3R',
  'borough': 'Queens',
  'neighborhood': 'Astoria',
  'extra': ['Exclusive', '650 ft²'],
  'sqft': 650},
 {'url': 'https://www.renthop.com/listings/185-claremont-avenue/4f/14509630',
  'rent': '2150',
  'bedrooms': 'Studio',
  'bath': '1',
  'address': '185 Claremont Avenue',
  'unit': 'Apt 4F',
  'borough': 'Manhattan',
  'neighborhood': 'Morningside Heights',
  'extra': ['Exclusive', 'Elevator'],
  'sqft': 0},
 {'url': 'https://www.renthop.com/listings/e-86th/9/14534348',
  'rent': '4400',
  'bedrooms': '3',
  'bath': '1.5',
  'address': ['E 86th'],
  'borough': 'Manhattan',
  'neighborhood': 'Yorkville',
  'extra': ['No Fee', 'Doorman', 'Elevator', 'Pre-War'],
  'sqft': 0},
 {'url': 'https://www.renthop.com/listings/324-east-52nd-street/b1/14523150',
  'rent': '3800',
  'bedrooms': '2',
  'bath': 